<a href="https://colab.research.google.com/github/Talizg03/weather/blob/main/4_2_25_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modify the code : YearMade(1000)
** Rows where YearMade == 1000** will have both:
YearMade set to NaN
** AgeOfVehicle set to NaN**
All other rows will have their normal age calculated based on the date_of_sale and YearMade.
This ensures that both the YearMade and AgeOfVehicle columns behave appropriately for the rows where YearMade is 1000.


In [1]:
import pandas as pd
df = pd.read_csv("Train.csv")

<ipython-input-1-97fbf718528a>:2: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Train.csv")


In [2]:
print(df.shape)

(401125, 53)


In [3]:
df2=df.copy()
df2.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
print(df2.shape)

(401125, 53)


In [5]:
import numpy as np
import pandas as pd

# Convert 'date_of_sale' to datetime format
df2['date_of_sale'] = pd.to_datetime(df2['saledate'], format='%m/%d/%Y %H:%M', errors='coerce')

# Set 'YearMade' to NaN if it's 1000
df2['YearMade'] = np.where(df2['YearMade'] == 1000, np.nan, df2['YearMade'])

# Add AgeOfVehicle column, with a condition to handle YearMade being NaN
df2['AgeOfVehicle'] = np.where(df2['YearMade'].isna(), np.nan, df2['date_of_sale'].dt.year - df2['YearMade'])

# Display the first few rows of the updated DataFrame
print(df2.head())





   SalesID  SalePrice  MachineID  ModelID  datasource  auctioneerID  YearMade  \
0  1139246      66000     999089     3157         121           3.0    2004.0   
1  1139248      57000     117657       77         121           3.0    1996.0   
2  1139249      10000     434808     7009         121           3.0    2001.0   
3  1139251      38500    1026470      332         121           3.0    2001.0   
4  1139253      11000    1057373    17311         121           3.0    2007.0   

   MachineHoursCurrentMeter UsageBand         saledate  ... Thumb  \
0                      68.0       Low  11/16/2006 0:00  ...   NaN   
1                    4640.0       Low   3/26/2004 0:00  ...   NaN   
2                    2838.0      High   2/26/2004 0:00  ...   NaN   
3                    3486.0      High   5/19/2011 0:00  ...   NaN   
4                     722.0    Medium   7/23/2009 0:00  ...   NaN   

  Pattern_Changer Grouser_Type Backhoe_Mounting Blade_Type Travel_Controls  \
0             NaN   

In [6]:
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
import pandas as pd

# turn all string/category columns into integers

for col in df2.select_dtypes(['object']):
    df2[col] = df[col].astype('category')
df2.info()

for col in df2.select_dtypes(['category']):
    df2[col] = df2[col].cat.codes

df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 55 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   SalesID                   401125 non-null  int64         
 1   SalePrice                 401125 non-null  int64         
 2   MachineID                 401125 non-null  int64         
 3   ModelID                   401125 non-null  int64         
 4   datasource                401125 non-null  int64         
 5   auctioneerID              380989 non-null  float64       
 6   YearMade                  362940 non-null  float64       
 7   MachineHoursCurrentMeter  142765 non-null  float64       
 8   UsageBand                 69639 non-null   category      
 9   saledate                  401125 non-null  category      
 10  fiModelDesc               401125 non-null  category      
 11  fiBaseModel               401125 non-null  category      
 12  fi

In [7]:
# create X and y
X = df2.drop(columns='SalePrice')
display(X.head())
y = df2['SalePrice']
display(y)

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,date_of_sale,AgeOfVehicle
0,1139246,999089,3157,121,3.0,2004.0,68.0,1,668,949,...,-1,-1,-1,-1,-1,-1,3,1,2006-11-16,2.0
1,1139248,117657,77,121,3.0,1996.0,4640.0,1,1722,1724,...,-1,-1,-1,-1,-1,-1,3,1,2004-03-26,8.0
2,1139249,434808,7009,121,3.0,2001.0,2838.0,0,1382,330,...,-1,-1,-1,-1,-1,-1,-1,-1,2004-02-26,3.0
3,1139251,1026470,332,121,3.0,2001.0,3486.0,0,2394,3673,...,-1,-1,-1,-1,-1,-1,-1,-1,2011-05-19,10.0
4,1139253,1057373,17311,121,3.0,2007.0,722.0,2,3179,4207,...,-1,-1,-1,-1,-1,-1,-1,-1,2009-07-23,2.0


,SalePrice
0,66000
1,57000
2,10000
3,38500
4,11000
...,...
401120,10500
401121,11000
401122,11500
401123,9000


In [8]:
# split into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.7)
X_train

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,date_of_sale,AgeOfVehicle
38463,1276922,1394823,6783,132,1.0,1991.0,NaN,-1,3682,981,...,-1,-1,-1,0,5,5,-1,-1,1997-09-18,6.0
215512,1680074,1342381,15148,132,4.0,NaN,NaN,-1,1472,2746,...,2,1,0,-1,-1,-1,-1,-1,2010-02-08,NaN
64285,1333014,1377089,1092,132,2.0,1994.0,NaN,-1,1427,475,...,2,1,0,-1,-1,-1,-1,-1,2002-02-04,8.0
40874,1281221,1246900,3170,132,2.0,1988.0,NaN,-1,1220,1056,...,-1,-1,-1,-1,-1,-1,-1,-1,1994-02-10,6.0
161494,1582538,1505530,4674,132,11.0,1988.0,NaN,-1,1731,906,...,2,1,0,-1,-1,-1,-1,-1,1997-03-27,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,1792573,1252444,9548,132,1.0,1989.0,NaN,-1,1735,1352,...,-1,-1,-1,-1,-1,-1,-1,-1,2003-03-27,14.0
365838,2554295,1550967,3857,149,1.0,1992.0,23244.0,2,1269,1749,...,-1,-1,-1,-1,-1,-1,3,1,2011-02-15,19.0
131932,1486248,428435,7256,132,2.0,2000.0,NaN,-1,998,1664,...,-1,-1,-1,-1,-1,-1,3,1,2006-12-16,6.0
146867,1518511,1276538,1638,132,2.0,1999.0,NaN,-1,2363,2308,...,-1,-1,-1,0,5,5,-1,-1,2003-05-17,4.0


In [9]:
# print the RMSE accuracy of the baseline (std dev)
print("RMSE Baseline accuracy:", y_test.std())

RMSE Baseline accuracy: 22932.4005340408


In [10]:
# create X and y
X = df2.drop(columns=['SalePrice','date_of_sale'])
display(X.head())
y = df2['SalePrice']
display(y)

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,AgeOfVehicle
0,1139246,999089,3157,121,3.0,2004.0,68.0,1,668,949,...,-1,-1,-1,-1,-1,-1,-1,3,1,2.0
1,1139248,117657,77,121,3.0,1996.0,4640.0,1,1722,1724,...,-1,-1,-1,-1,-1,-1,-1,3,1,8.0
2,1139249,434808,7009,121,3.0,2001.0,2838.0,0,1382,330,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,3.0
3,1139251,1026470,332,121,3.0,2001.0,3486.0,0,2394,3673,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,10.0
4,1139253,1057373,17311,121,3.0,2007.0,722.0,2,3179,4207,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,2.0


,SalePrice
0,66000
1,57000
2,10000
3,38500
4,11000
...,...
401120,10500
401121,11000
401122,11500
401123,9000


In [11]:
# split into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.7)
X_train

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,AgeOfVehicle
38463,1276922,1394823,6783,132,1.0,1991.0,NaN,-1,3682,981,...,-1,-1,-1,-1,0,5,5,-1,-1,6.0
215512,1680074,1342381,15148,132,4.0,NaN,NaN,-1,1472,2746,...,28,2,1,0,-1,-1,-1,-1,-1,NaN
64285,1333014,1377089,1092,132,2.0,1994.0,NaN,-1,1427,475,...,28,2,1,0,-1,-1,-1,-1,-1,8.0
40874,1281221,1246900,3170,132,2.0,1988.0,NaN,-1,1220,1056,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,6.0
161494,1582538,1505530,4674,132,11.0,1988.0,NaN,-1,1731,906,...,28,2,1,0,-1,-1,-1,-1,-1,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,1792573,1252444,9548,132,1.0,1989.0,NaN,-1,1735,1352,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,14.0
365838,2554295,1550967,3857,149,1.0,1992.0,23244.0,2,1269,1749,...,-1,-1,-1,-1,-1,-1,-1,3,1,19.0
131932,1486248,428435,7256,132,2.0,2000.0,NaN,-1,998,1664,...,-1,-1,-1,-1,-1,-1,-1,3,1,6.0
146867,1518511,1276538,1638,132,2.0,1999.0,NaN,-1,2363,2308,...,-1,-1,-1,-1,0,5,5,-1,-1,4.0


In [ ]:
# print the RMSE accuracy of the baseline (std dev)
print("RMSE Baseline accuracy:", y_test.std())

RMSE Baseline accuracy: 22932.4005340408


In [12]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100,random_state=42, n_jobs=-1)
model = model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

def RMSE(y_, y_pred_):
    return ((y_ - y_pred_) ** 2).mean() ** 0.5

print(f'Train RMSE: {RMSE(y_train, y_train_pred):.3f}')
print(f'Test RMSE: {RMSE(y_test, y_test_pred):.3f}')

Train RMSE: 2783.208
Test RMSE: 7386.465


In [15]:
# Train the Isolation Forest model
from sklearn.ensemble import IsolationForest
iso_forest = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
iso_forest.fit(X_train)

# Predict anomaly scores and labels on a copy of X_test
X_test_copy = X_test.copy()
df2['anomaly_score'] = iso_forest.decision_function(X)  # Quantitative weirdness
df2['anomaly'] = iso_forest.predict(X)  # Binary anomaly label
df2['anomaly_label'] = df2['anomaly'].map({1: 'Normal', -1: 'Anomaly'})

In [16]:
df2.anomaly.value_counts()

,count
anomaly,
1,397072
-1,4053


In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

model = RandomForestRegressor()
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print("Test RMSE:", mean_squared_error(y_test, y_test_pred) ** 0.5)

Test RMSE: 7392.79097627605


In [19]:
df3=df2.copy()
df3.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,date_of_sale,AgeOfVehicle,anomaly_score,anomaly,anomaly_label
0,1139246,66000,999089,3157,121,3.0,2004.0,68.0,1,668,...,-1,-1,-1,3,1,2006-11-16,2.0,0.095856,1,Normal
1,1139248,57000,117657,77,121,3.0,1996.0,4640.0,1,1722,...,-1,-1,-1,3,1,2004-03-26,8.0,0.071891,1,Normal
2,1139249,10000,434808,7009,121,3.0,2001.0,2838.0,0,1382,...,-1,-1,-1,-1,-1,2004-02-26,3.0,0.110645,1,Normal
3,1139251,38500,1026470,332,121,3.0,2001.0,3486.0,0,2394,...,-1,-1,-1,-1,-1,2011-05-19,10.0,0.073740,1,Normal
4,1139253,11000,1057373,17311,121,3.0,2007.0,722.0,2,3179,...,-1,-1,-1,-1,-1,2009-07-23,2.0,0.062220,1,Normal


In [20]:
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

for col in df3.select_dtypes(['object']):
    df3[col] = df2[col].astype('category')
df3.info()

for col in df3.select_dtypes(['category']):
    df3[col] = df3[col].cat.codes
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 58 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   SalesID                   401125 non-null  int64         
 1   SalePrice                 401125 non-null  int64         
 2   MachineID                 401125 non-null  int64         
 3   ModelID                   401125 non-null  int64         
 4   datasource                401125 non-null  int64         
 5   auctioneerID              380989 non-null  float64       
 6   YearMade                  362940 non-null  float64       
 7   MachineHoursCurrentMeter  142765 non-null  float64       
 8   UsageBand                 401125 non-null  int8          
 9   saledate                  401125 non-null  int16         
 10  fiModelDesc               401125 non-null  int16         
 11  fiBaseModel               401125 non-null  int16         
 12  fi

In [21]:
print(df3.shape)

(401125, 58)


In [22]:
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

for col in df3.select_dtypes(['object']):
    df3[col] = df2[col].astype('category')


for col in df3.select_dtypes(['category']):
    df3[col] = df3[col].cat.codes
df3 = df3[df2.anomaly==1].copy()
X2 = df3.drop(columns=['SalePrice', 'anomaly_label', 'anomaly', 'anomaly_score','date_of_sale'])
X2 = pd.get_dummies(X2, drop_first=True)
y2 = df3['SalePrice']
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state=42)

model2 = RandomForestRegressor()
model2.fit(X2, y2)
y2_test_pred = model2.predict(X2_test)
y2_train_pred = model2.predict(X2_train)
def RMSE(y2_, y2_pred_):
    return ((y2_ - y2_pred_) ** 2).mean() ** 0.5

print(f'Train RMSE: {RMSE(y2_train, y2_train_pred):.3f}')
print(f'Test RMSE: {RMSE(y2_test, y2_test_pred):.3f}')

Train RMSE: 2689.940
Test RMSE: 2678.495


In [23]:
import joblib
model_data2 = [model2, iso_forest]
joblib.dump(model_data2, '/content/4.2.25_#3.pkl')  # Save directly to the file

['/content/4.2.25_#3.pkl']

In [24]:
import pandas as pd
df_valid = pd.read_csv("Valid.csv")

In [26]:
import numpy as np
import pandas as pd

# Convert 'date_of_sale' to datetime format
df_valid['date_of_sale'] = pd.to_datetime(df_valid['saledate'], format='%m/%d/%Y %H:%M', errors='coerce')

# Set 'YearMade' to NaN if it's 1000
df_valid['YearMade'] = np.where(df_valid['YearMade'] == 1000, np.nan, df_valid['YearMade'])

# Add AgeOfVehicle column, with a condition to handle YearMade being NaN
df_valid['AgeOfVehicle'] = np.where(df_valid['YearMade'].isna(), np.nan, df_valid['date_of_sale'].dt.year - df_valid['YearMade'])

# Display the first few rows of the updated DataFrame
print(df_valid.head())


   SalesID  MachineID  ModelID  datasource  auctioneerID  YearMade  \
0  1222837     902859     1376         121             3       NaN   
1  1222839    1048320    36526         121             3    2006.0   
2  1222841     999308     4587         121             3    2000.0   
3  1222843    1062425     1954         121             3       NaN   
4  1222845    1032841     4701         121             3    2002.0   

   MachineHoursCurrentMeter UsageBand       saledate fiModelDesc  ...  \
0                       0.0       NaN  1/5/2012 0:00        375L  ...   
1                    4412.0    Medium  1/5/2012 0:00    TX300LC2  ...   
2                   10127.0    Medium  1/5/2012 0:00       270LC  ...   
3                    4682.0       Low  1/5/2012 0:00      892DLC  ...   
4                    8150.0    Medium  1/4/2012 0:00        544H  ...   

                 Thumb      Pattern_Changer Grouser_Type Backhoe_Mounting  \
0  None or Unspecified  None or Unspecified       Double       

In [28]:
print(df_valid.shape)

(11573, 54)


In [29]:
print(df_valid.columns)

Index(['SalesID', 'MachineID', 'ModelID', 'datasource', 'auctioneerID',
       'YearMade', 'MachineHoursCurrentMeter', 'UsageBand', 'saledate',
       'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries',
       'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'state',
       'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure',
       'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission',
       'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type',
       'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier',
       'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System',
       'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type',
       'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer',
       'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'date_of_sale',
       'AgeOfVehicle'],
      dtype='object')


In [30]:
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

for col in df_valid.select_dtypes(['object']):
    df_valid[col] = df_valid[col].astype('category')

for col in df_valid.select_dtypes(['category']):
    df_valid[col] = df_valid[col].cat.codes
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11573 entries, 0 to 11572
Data columns (total 54 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   SalesID                   11573 non-null  int64         
 1   MachineID                 11573 non-null  int64         
 2   ModelID                   11573 non-null  int64         
 3   datasource                11573 non-null  int64         
 4   auctioneerID              11573 non-null  int64         
 5   YearMade                  10367 non-null  float64       
 6   MachineHoursCurrentMeter  4739 non-null   float64       
 7   UsageBand                 11573 non-null  int8          
 8   saledate                  11573 non-null  int8          
 9   fiModelDesc               11573 non-null  int16         
 10  fiBaseModel               11573 non-null  int16         
 11  fiSecondaryDesc           11573 non-null  int8          
 12  fiModelSeries     

In [31]:
print(X2.shape)  # Shape of the data you're passing to decision_function
print(df_valid.shape)  # Shape of the DataFrame you're adding the 'anomaly_score' to

(397072, 53)
(11573, 54)


In [32]:
df_valid=df_valid.drop(columns=['date_of_sale'])
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11573 entries, 0 to 11572
Data columns (total 53 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SalesID                   11573 non-null  int64  
 1   MachineID                 11573 non-null  int64  
 2   ModelID                   11573 non-null  int64  
 3   datasource                11573 non-null  int64  
 4   auctioneerID              11573 non-null  int64  
 5   YearMade                  10367 non-null  float64
 6   MachineHoursCurrentMeter  4739 non-null   float64
 7   UsageBand                 11573 non-null  int8   
 8   saledate                  11573 non-null  int8   
 9   fiModelDesc               11573 non-null  int16  
 10  fiBaseModel               11573 non-null  int16  
 11  fiSecondaryDesc           11573 non-null  int8   
 12  fiModelSeries             11573 non-null  int8   
 13  fiModelDescriptor         11573 non-null  int8   
 14  Produc

In [33]:
print(X2.shape)  # Shape of the data you're passing to decision_function
print(df_valid.shape)  # Shape of the DataFrame you're adding the 'anomaly_score' to

(397072, 53)
(11573, 53)


In [34]:
print(df_valid.columns)  # List all column names in df_valid
print(X2.columns)  # List all column names in X2


Index(['SalesID', 'MachineID', 'ModelID', 'datasource', 'auctioneerID',
       'YearMade', 'MachineHoursCurrentMeter', 'UsageBand', 'saledate',
       'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries',
       'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'state',
       'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure',
       'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission',
       'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type',
       'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier',
       'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System',
       'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type',
       'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer',
       'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'AgeOfVehicle'],
      dtype='object')
Index(['SalesID', 'MachineID', 'ModelID', 'datasour

In [ ]:
# Save features used during training
#training_features = df_train.columns

# Apply the same set of features to the validation data

In [35]:
df_valid_processed = df_valid.drop(columns=['anomaly_score', 'anomaly', 'anomaly_label', 'date_of_sale'], errors='ignore')
df_valid_processed = pd.get_dummies(df_valid_processed, drop_first=True)

# Ensure columns match training features
df_valid_processed = df_valid_processed.reindex(columns=X2.columns, fill_value=0)

In [36]:
# Compute anomaly scores and predictions
df_valid['anomaly_score'] = iso_forest.decision_function(df_valid_processed)
df_valid['anomaly'] = iso_forest.predict(df_valid_processed)

# Map predictions to labels
df_valid['anomaly_label'] = df_valid['anomaly'].map({1: 'Normal', -1: 'Anomaly'})

In [37]:
# Summary of anomalies
anomaly_counts = df_valid['anomaly_label'].value_counts()
print(anomaly_counts)

# Inspect a few rows with anomalies
anomalies = df_valid[df_valid['anomaly_label'] == 'Anomaly']
print(anomalies.head())

anomaly_label
Normal     10788
Anomaly      785
Name: count, dtype: int64
    SalesID  MachineID  ModelID  datasource  auctioneerID  YearMade  \
8   1222855     531393    23926         121             3       NaN   
42  1223057    1011051    23931         121             3    2003.0   
43  1223058    1035918    23929         121             3    1999.0   
44  1223059    1006893    22358         121             3    2005.0   
45  1223060    1036752    23931         121             3    2003.0   

    MachineHoursCurrentMeter  UsageBand  saledate  fiModelDesc  ...  \
8                     8145.0          1        18           22  ...   
42                    5766.0          2         8           36  ...   
43                    7863.0          2         8           30  ...   
44                    4348.0          2         8          677  ...   
45                   10484.0          0         8           36  ...   

    Grouser_Type  Backhoe_Mounting  Blade_Type  Travel_Controls  \
8    

In [38]:
print(X2.shape)  # Shape of the data you're passing to decision_function
print(df_valid.shape)  # Shape of the DataFrame you're adding the 'anomaly_score' to

(397072, 53)
(11573, 56)


In [39]:
df_valid=df_valid.drop(columns=['anomaly_score','anomaly','anomaly_label'])
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11573 entries, 0 to 11572
Data columns (total 53 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SalesID                   11573 non-null  int64  
 1   MachineID                 11573 non-null  int64  
 2   ModelID                   11573 non-null  int64  
 3   datasource                11573 non-null  int64  
 4   auctioneerID              11573 non-null  int64  
 5   YearMade                  10367 non-null  float64
 6   MachineHoursCurrentMeter  4739 non-null   float64
 7   UsageBand                 11573 non-null  int8   
 8   saledate                  11573 non-null  int8   
 9   fiModelDesc               11573 non-null  int16  
 10  fiBaseModel               11573 non-null  int16  
 11  fiSecondaryDesc           11573 non-null  int8   
 12  fiModelSeries             11573 non-null  int8   
 13  fiModelDescriptor         11573 non-null  int8   
 14  Produc

In [ ]:
print(X2.shape)  # Shape of the data you're passing to decision_function
print(df_valid.shape)  # Shape of the DataFrame you're adding the 'anomaly_score' to

In [2]:
import joblib
from sklearn.ensemble import IsolationForest


# Step 1: Load the entire saved model (both Isolation Forest and main model)
model_data2 = joblib.load('/content/4.2.25_#3.pkl')

# Access the models by index (not by string key)
model2 = model_data2[0]  # First element in the list (RandomForestRegressor)
isolation_forest = model_data2[1]  # Second element in the list (Isolation Forest model)

#Step 3: Apply Isolation Forest to detect anomalies in new data (new_df)
#outliers_new = isolation_forest.predict(df_valid)

#Step 4: Keep only the normal rows (where prediction is 1)
#new_df_clean = df_valid[outliers_new == 1]

# Step 5: Make predictions using the main model (RandomForestRegressor)
y2_pred_ = model2.predict(df_valid)

# Step 6: Optionally, add the predictions to the cleaned dataframe
df_valid['y2_pred_'] = y2_pred_

# Step 7: View the predictions
print(df_valid.head())


NameError: name 'df_valid' is not defined

In [1]:
print(df_valid.shape)

NameError: name 'df_valid' is not defined

In [ ]:
df_valid.to_csv('clean_pred_V2.csv', index=False)